In [1]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd


models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/fastspeech2-en-ljspeech",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)


2022-07-26 00:49:30 | INFO | fairseq.tasks.speech_to_text | dictionary size (vocab.txt): 75


In [8]:
model = models[0]
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)
generator = task.build_generator(models, cfg)

text = "Privet Drive was a perfectly normal street. It was lined with perfectly normal houses, one nearly identical to the next. The inhabitants of these homes were all perfectly normal people as well, save for one house. In this house lived a family who tried its hardest to be perfectly normal. But there has always been one aspect of Mr. and Mrs. Dursley's lives that has never been anything close to their definition of normal. That thing was sitting on his lumpy bed in the smallest room of the house, filled with their son's old, broken toys accumulated throughout his pampered life. That thing was none other than their nephew, Harry Potter."

model = model.to("cuda:0")

sample = TTSHubInterface.get_model_input(task, text)
sample["net_input"]["src_tokens"] = sample["net_input"]["src_tokens"].to("cuda:0")
sample["net_input"]["src_lengths"] = sample["net_input"]["src_lengths"].to("cuda:0")
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)


2022-07-26 00:55:16 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from C:/Users/ryanr/.cache/fairseq/models--facebook--fastspeech2-en-ljspeech/snapshots/a3e3e5e2e62bb7ca7514b11aa469e9c5b01a20bf/hifigan.bin


In [ ]:
import numpy as np
from scipy.io.wavfile import write

data = wav.cpu().numpy() # 44100 random samples between -1 and 1
scaled = np.int16(data/np.max(np.abs(data)) * 32767)
write('test.wav', rate, scaled)